In [1]:
import random

import numpy as np
import torch
from torch.utils.data.dataloader import default_collate

from settings import EXPERIMENTS_DIR
from experiment import Experiment
from utils import to_device, load_weights, load_embeddings, create_embeddings_matrix
from vocab import Vocab
from train import create_model
from preprocess_train import load_dataset, create_dataset_reader

In [87]:
exp_id = './train.bfxkyc9m'

# Load everything

In [88]:
exp = Experiment.load(EXPERIMENTS_DIR, exp_id)

In [89]:
exp

In [90]:
exp.config.preprocess_exp_id

'preprocess.gb54gqgr'

In [91]:
preprocess_exp = Experiment.load(EXPERIMENTS_DIR, exp.config.preprocess_exp_id)
# preprocess_exp = Experiment.load(EXPERIMENTS_DIR, "preprocess.c31qq46k")
dataset_train, dataset_val, dataset_test, vocab, style_vocab, W_emb = load_dataset(preprocess_exp)

In [92]:
dataset_reader = create_dataset_reader(preprocess_exp.config)

In [93]:
model = create_model(exp.config, vocab, style_vocab, dataset_train.max_len, W_emb)

In [94]:
load_weights(model, exp.experiment_dir.joinpath('best.th'))

In [95]:
model = model.eval()

## Predict

In [96]:
def create_inputs(instances):
    if not isinstance(instances, list):
        instances = [instances,]
        
    if not isinstance(instances[0], dict):
        sentences = [
            dataset_reader.preprocess_sentence(dataset_reader.spacy( dataset_reader.clean_sentence(sent)))
            for sent in instances
        ]
        
        style = list(style_vocab.token2id.keys())[0]
        instances = [
            {
                'sentence': sent,
                'style': style,
            }
            for sent in sentences
        ]
        
        for inst in instances:
            inst_encoded = dataset_train.encode_instance(inst)
            inst.update(inst_encoded)            
    
    
    instances = [
        {
            'sentence': inst['sentence_enc'],
            'style': inst['style_enc'],
        } 
        for inst in instances
    ]
    
    instances = default_collate(instances)
    instances = to_device(instances)      
    
    return instances

In [97]:
def get_sentences(outputs):
    predicted_indices = outputs["predictions"]
    end_idx = vocab[Vocab.END_TOKEN]
    
    if not isinstance(predicted_indices, np.ndarray):
        predicted_indices = predicted_indices.detach().cpu().numpy()

    all_predicted_tokens = []
    for indices in predicted_indices:
        indices = list(indices)

        # Collect indices till the first end_symbol
        if end_idx in indices:
            indices = indices[:indices.index(end_idx)]

        predicted_tokens = [vocab.id2token[x] for x in indices]
        all_predicted_tokens.append(predicted_tokens)
        
    return all_predicted_tokens

In [98]:
sentence =  ' '.join(dataset_val.instances[1]['sentence'])

In [99]:
sentence

'） 平成 24 年 7 月 、 入管 法 が 変わり ます ！ 詳しく は 、 こちら へ 。'

In [100]:
inputs = create_inputs(sentence)

In [101]:
outputs = model(inputs)

In [102]:
sentences = get_sentences(outputs)

In [103]:
' '.join(sentences[0])

''

### Swap style

In [104]:
possible_styles = list(style_vocab.token2id.keys()) #['negative', 'positive']

In [105]:
possible_styles

['keigo', 'normal']

In [106]:
sentences0 = [s for s in dataset_val.instances if s['style'] == possible_styles[0]]
sentences1 = [s for s in dataset_val.instances if s['style'] == possible_styles[1]]

In [107]:
for i in np.random.choice(np.arange(len(sentences0)), 5):
    print(i, ' '.join(sentences0[i]['sentence']))

22 「 スマートフォン を ご 利用 の 皆 さま は ぜひ ご 活用 いただけれ ば 」 と 同局 担当 者
27 それ で は 、 「 お 招き 頂き まし て ありがとう ござい ます 」 、 「 喜ん で 出席
2 この 証拠 は 、 お 客 様 の ベスト 1 日 ツアー 1 日 ツアー ベスト を 支払っ て
34 クリエイター 検定 の 学習 に 要求 さ れる こと は 才能 より も これ が 楽しい と 思う こと
24 その こと を お 手伝い する の が 当科 の 役目 です 。


In [108]:
for i in np.random.choice(np.arange(len(sentences1)), 5):
    print(i, ' '.join(sentences1[i]['sentence']))

3 あなた たち の よう に 、 この 時期 おいしい もの も 多い し 、 つい 食べ 過ぎ て 太っ
6 今回 は 、 この 方 の 転職 先 と お 仕事 が でき ない か 、 と いう こと
10 で も 、 私 って ば 自分 で 自分 の 家計 を 動かし て いる 意識 が 低い の
5 日本 軍 に 勝利 し た もの の 、 イギリス は 独立 を 許さ ず 、 再び イギリス 領
4 はじめて 取得 し た 人 は 、 「 モナコ グランプリ に 参戦 する 」 でし た 。


#### Swap

In [109]:
target0 = np.random.choice(np.arange(len(sentences0)))
target1 = np.random.choice(np.arange(len(sentences1)))

In [110]:
print(' '.join(sentences0[target0]['sentence']))

多少 時間 を もっ て 登れ ば どなた に も 登れ ます 。


In [111]:
print(' '.join(sentences1[target1]['sentence']))

今 ある 安 万年 筆 と し て は これ 、 かなり いい ほう か と 。


In [112]:
inputs = create_inputs([
    sentences0[target0],
    sentences1[target1],
])

In [113]:
z_hidden = model(inputs)

In [114]:
z_hidden['style_hidden'].shape

torch.Size([2, 128])

In [115]:
z_hidden['meaning_hidden'].shape

torch.Size([2, 128])

In [116]:
original_decoded = model.decode(z_hidden)

In [117]:
original_sentences = get_sentences(original_decoded)

In [118]:
print(' '.join(original_sentences[0]))
print(' '.join(original_sentences[1]))

In [57]:
z_hidden_swapped = {
    'meaning_hidden': torch.stack([
        z_hidden['meaning_hidden'][0].clone(),
        z_hidden['meaning_hidden'][1].clone(),        
    ], dim=0),
    'style_hidden': torch.stack([
        z_hidden['style_hidden'][1].clone(),
        z_hidden['style_hidden'][0].clone(),        
    ], dim=0),
}

In [58]:
swaped_decoded = model.decode(z_hidden_swapped)

In [59]:
swaped_sentences = get_sentences(swaped_decoded)

In [60]:
print(' '.join(original_sentences[0]))
print(' '.join(original_sentences[1]))
print()
print(' '.join(swaped_sentences[0]))
print(' '.join(swaped_sentences[1]))

お の の の の の に て 、 、 、 の の の の の の
お の の の の の に て 、 、 、 の の の の の の

お の の の の の に て 、 、 、 の の の の の の
お の の の の の に て 、 、 、 の の の の の の
